<a href="https://colab.research.google.com/github/BenFeng666/Testing_1/blob/main/testing_9-26.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install compatible stack for torch 2.8.0
!pip install -q bitsandbytes==0.42.0
!pip install -q triton==3.4.0
!pip install -q accelerate transformers einops
!pip install -q torchvision


In [ ]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 139.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.1
    Uninstalling transformers-4.56.1:
      Successfully uninstalled transformers-4.56.1


In [ ]:
!pip uninstall -y torch torchvision torchaudio bitsandbytes
!pip install torch==2.1.2+cu118 torchvision==0.16.2+cu118 torchaudio==2.1.2+cu118 -f https://download.pytorch.org/whl/torch_stable.html
!pip install bitsandbytes==0.43.1



Found existing installation: torch 2.8.0
Uninstalling torch-2.8.0:
  Successfully uninstalled torch-2.8.0
Found existing installation: torchvision 0.23.0
Uninstalling torchvision-0.23.0:
  Successfully uninstalled torchvision-0.23.0
Found existing installation: bitsandbytes 0.42.0
Uninstalling bitsandbytes-0.42.0:
  Successfully uninstalled bitsandbytes-0.42.0
Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==2.1.2+cu118 (from versions: 2.2.0, 2.2.0+cpu, 2.2.0+cpu.cxx11.abi, 2.2.0+cu118, 2.2.0+cu121, 2.2.0+rocm5.6, 2.2.0+rocm5.7, 2.2.1, 2.2.1+cpu, 2.2.1+cpu.cxx11.abi, 2.2.1+cu118, 2.2.1+cu121, 2.2.1+rocm5.6, 2.2.1+rocm5.7, 2.2.2, 2.2.2+cpu, 2.2.2+cpu.cxx11.abi, 2.2.2+cu118, 2.2.2+cu121, 2.2.2+rocm5.6, 2.2.2+rocm5.7, 2.3.0, 2.3.0+cpu, 2.3.0+cpu.cxx11.abi, 2.3.0+cu118, 2.3.0+cu121, 2.3.0+rocm5.7, 2.3.0+rocm6.0, 2.3.1, 2.3.1+cpu, 2.3.1+cpu.cxx11.abi, 2.3.1+cu118, 2.3.1+cu121, 2.3.1+rocm5.7, 2.3.1+rocm

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch

model_name_or_id = "AI4Chem/ChemLLM-7B-Chat"

# Must be True, otherwise the model won't load
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_id,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_id, trust_remote_code=True)

prompt = "What is Molecule of Ibuprofen?"
enc = tokenizer(prompt, return_tensors="pt").to("cuda")

# Custom lightweight generate loop (avoids buggy past_key_values)
def simple_generate(model, tokenizer, enc, max_new_tokens=200):
    model.eval()
    input_ids = enc["input_ids"].clone()
    attention_mask = enc["attention_mask"].clone()

    for _ in range(max_new_tokens):
        with torch.inference_mode():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits[:, -1, :]
            next_token = torch.argmax(logits, dim=-1).unsqueeze(-1)

        input_ids = torch.cat([input_ids, next_token], dim=-1)
        attention_mask = torch.cat(
            [attention_mask, torch.ones_like(next_token)], dim=-1
        )

        if next_token.item() == tokenizer.eos_token_id:
            break

    return tokenizer.decode(input_ids[0], skip_special_tokens=True)

# Run it
output_text = simple_generate(model, tokenizer, enc, max_new_tokens=200)
print(output_text)







/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/936 [00:00<?, ?B/s]

configuration_internlm.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/AI4Chem/ChemLLM-7B-Chat:
- configuration_internlm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`torch_dtype` is deprecated! Use `dtype` instead!


modeling_internlm2.py: 0.00B [00:00, ?B/s]

...

tokenization_internlm.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/AI4Chem/ChemLLM-7B-Chat:
- tokenization_internlm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


./tokenizer.model:   0%|          | 0.00/1.48M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

What is Molecule of Ibuprofen? – Molecule of Ibuprofen is the chemical compound of C13H18O2. It is a non-steroidal anti-inflammatory drug (NSAID) used to treat pain, fever, and inflammation. It is also used to treat menstrual cramps, migraines, and rheumatoid arthritis. Ibuprofen is available as over-the-counter (OTC) medication, as well as by prescription. It is available in various forms, including tablets, capsules, liquid, and suppositories. Ibuprofen is a member of the class of drugs known as non-steroidal anti-inflammatory drugs (NSAIDs). NSAIDs are used to treat pain, inflammation, and fever. They work by blocking the production of prostaglandins, which are chemicals that cause pain, inflammation, and fever. Ibuprofen is one of the most widely used NSAIDs. It is available as over-the-counter (OT


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from google.colab import files
import json
import numpy as np

# === 1) Upload file ===
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# === 2) Read Excel file ===
df = pd.read_excel(filename, header=0)
df.columns = df.columns.str.strip()

print("Columns found:", df.columns.tolist())

# === 3) Keep only smiles + mTP ===
df = df[["smiles", "mTP"]].rename(columns={
    "smiles": "smiles",
    "mTP": "score"
})

# === 4) Drop missing values ===
df = df.dropna().reset_index(drop=True)
print("Rows after cleaning:", len(df))

# === 5) Shuffle scores completely random ===
shuffled_scores = df["score"].sample(frac=1, random_state=42).reset_index(drop=True)
df["score"] = shuffled_scores

# === 6) Train-test split (80/20) ===
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)
train_df, test_df = train_df.reset_index(drop=True), test_df.reset_index(drop=True)

print("Train size:", len(train_df), " Test size:", len(test_df))

# === 7) Convert to JSON ===
train_json = [
    {"id": str(i+1), "smiles": r.smiles, "score": float(r.score)}
    for i, r in train_df.iterrows()
]
test_json = [
    {"id": str(i+1), "smiles": r.smiles, "score": float(r.score)}
    for i, r in test_df.iterrows()
]

# === 8) Save JSON files ===
with open("train.json", "w") as f:
    json.dump(train_json, f, indent=2)
with open("test.json", "w") as f:
    json.dump(test_json, f, indent=2)

print("✅ Finished! Files saved: train.json, test.json")
print("Sample train:", train_json[:2])
print("Sample test:", test_json[:2])







Saving NC-data for machine learning.xlsx to NC-data for machine learning.xlsx
Columns found: ['number', nan, nan, nan, 'smiles', 'mTP']
Rows after cleaning: 1200
Train size: 960  Test size: 240
✅ Finished! Files saved: train.json, test.json
Sample train: [{'id': '1', 'smiles': 'CCCCCCCCCCCCCCCCNC(=O)C(CCCCCOC(=O)CCCCCCCCC=C)NCCCN(C)C', 'score': 2.37490674}, {'id': '2', 'smiles': 'CCCCCCCC\\C=C/CCCCCCCCNC(=O)C(CCCCCOC(=O)CCC(C)CCCCC)NCCCN1CCOCC1', 'score': 6.7891369}]
Sample test: [{'id': '1', 'smiles': 'CCCCCCCCCCCCCCCCCCNC(=O)C(CCCCCOC(=O)CCCCCCCCC)NC1CCNCC1', 'score': 5.64996478}, {'id': '2', 'smiles': 'CCCCCCCCCCCCCCCCNC(=O)C(CCCCCOC(=O)CCC(C)CCCCC)NCCN1CCCCC1', 'score': 6.58906319}]


In [ ]:
SCORE_SCALE = """
Score reference table:
0 – Slight mRNA transfection efficiency from crude lipid library (0–0.1 fold vs commercial lipid, in vitro data).
1 – Slight mRNA transfection efficiency from purified lipid library (0–0.1 fold vs commercial lipid, in vitro data).
2 – Obvious mRNA transfection efficiency from crude lipid library, less than commercial lipid (0.1–1 fold, in vitro).
3 – Obvious mRNA transfection efficiency from purified lipid library, less than commercial lipid (0.1–1 fold, in vitro).
4 – Obvious mRNA transfection efficiency from crude lipid library, better than commercial lipid (>1 fold), not final candidates (no purified lipid data).
5 – Obvious mRNA transfection efficiency from purified lipid library, better than commercial lipid (>1 fold, in vitro).
6 – Top lipid for in vivo study, not better than commercial lipids (MC3, ALC-0315, SM102).
7 – Top lipid for in vivo study, better than MC3 and ALC-0315.
8 – Top lipid for in vivo study, better than SM102.
"""


In [ ]:
import torch, json, re
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# ==== 1. Define system prompt with examples ====
K = 150
few_examples = train_json[:K]

SYSTEM_PROMPT = (
    "You are a medicinal chemistry and drug delivery assistant.\n\n"
    "Context:\n"
    "I will provide training examples with {\"smiles\": \"...\", \"score\": <float>}.\n\n"
    f"Training examples:\n{json.dumps(few_examples, indent=2)}\n\n"
    "Task:\n"
    "Given the following target SMILES, predict:\n"
    "1. A numeric score (float).\n"
    "2. A confidence value (1–10).\n\n"
    "IMPORTANT OUTPUT RULES:\n"
    "– Line 1: ONLY the score (float).\n"
    "– Line 2: ONLY the confidence (1–10).\n"
    "– Do NOT write JSON, code, or explanations.\n"
)

# ==== 2. Helper: build model inputs ====
def build_inputs(smiles: str):
    try:
        prompt_text = tokenizer.apply_chat_template(
            [
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": f"SMILES: {smiles}"}
            ],
            add_generation_prompt=True,
            tokenize=False
        )
    except (ValueError, AttributeError):
        prompt_text = SYSTEM_PROMPT + f"\n\nSMILES: {smiles}\n"

    enc = tokenizer(prompt_text, return_tensors="pt")
    enc = {k: v.to("cuda") for k, v in enc.items()}

    if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = tokenizer.eos_token_id
    if getattr(model.generation_config, "pad_token_id", None) is None:
        model.generation_config.pad_token_id = tokenizer.eos_token_id
    return enc

# ==== 3. Lightweight generate ====
def simple_generate(model, tokenizer, enc, max_new_tokens=50):
    model.eval()
    input_ids = enc["input_ids"].clone()
    attention_mask = enc["attention_mask"].clone()

    for _ in range(max_new_tokens):
        with torch.inference_mode():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits[:, -1, :]
            next_token = torch.argmax(logits, dim=-1).unsqueeze(-1)

        input_ids = torch.cat([input_ids, next_token], dim=-1)
        attention_mask = torch.cat(
            [attention_mask, torch.ones_like(next_token)], dim=-1
        )

        if next_token.item() == tokenizer.eos_token_id:
            break

    # ✅ Only decode the continuation
    new_tokens = input_ids[0][enc["input_ids"].shape[1]:]
    gen_text = tokenizer.decode(new_tokens, skip_special_tokens=True).strip()
    return gen_text

# ==== 4. Run test predictions ====
for i, sample in enumerate(test_json[:50], start=1):
    query_smiles = sample["smiles"]
    true_score = sample["score"]

    enc = build_inputs(query_smiles)
    gen_text = simple_generate(model, tokenizer, enc, max_new_tokens=50)

    # 🔍 Find first float anywhere in output
    m = re.search(r"[-+]?\d*\.\d+|\d+", gen_text)
    pred_eff = float(m.group(0)) if m else None

    # 🔍 Find confidence as integer 1–10
    m_conf = re.search(r"\b([1-9]|10)\b", gen_text)
    pred_conf = int(m_conf.group(1)) if m_conf else None

    # === Print result ===
    print(f"\n=== Sample {i} ===")
    print("SMILES:", query_smiles)
    print(f"Predicted score:      {pred_eff}")
    print(f"Actual score:         {true_score}")
    print(f"Confidence (1–10):    {pred_conf}")
    print("Raw output:", gen_text)




=== Sample 1 ===
SMILES: CCCCCCCCCCCCCCCCCCNC(=O)C(CCCCCOC(=O)CCCCCCCCC)NC1CCNCC1
Predicted score:      2.37496737402344
Actual score:         5.64996478
Confidence (1–10):    2
Raw output: [{
  "score": 2.37496737402344,
  "confidence": 1
}]

=== Sample 2 ===
SMILES: CCCCCCCCCCCCCCCCNC(=O)C(CCCCCOC(=O)CCC(C)CCCCC)NCCN1CCCCC1
Predicted score:      2.37490674
Actual score:         6.58906319
Confidence (1–10):    2
Raw output: [{
  "score": 2.37490674,
  "confidence": 1
}]

=== Sample 3 ===
SMILES: CCCCCCCCCCCCCCCCCCNC(=O)C(CCCCCOC(=O)CCCCCCCC)NCCCN1CCCC1
Predicted score:      2.37496737402344
Actual score:         4.66712716
Confidence (1–10):    2
Raw output: [{
  "score": 2.37496737402344,
  "confidence": 1
}]

=== Sample 4 ===
SMILES: CCCCCCCCCCCCCCNC(=O)C(CCCCCOC(=O)CCCCCCCCC=C)NCCCN(CCCC)CCCC
Predicted score:      0.0
Actual score:         9.95198523
Confidence (1–10):    1
Raw output: [{
  "score": 0.0,
  "confidence": 1
}]

=== Sample 5 ===
SMILES: CCCCCCCC\C=C/CCCCCCCCNC(=O)C(

KeyboardInterrupt: 